# Prompt2Code with reflexion - SINGLE CASE TEST

Instead of generate the answer, prompt to generate the function

In [158]:
import os
import re
import string
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np

In [159]:
# Read the CSV file into a Pandas DataFrame
# print(os.getcwd())
calendar_data = pd.read_csv("../data/calendar_data.csv")
print(f"The number of calendar data is {len(calendar_data)}.")

with open("../data/question.json") as json_file:
    json_data = json.load(json_file)
    # print(json_data[question_index])

print(f"The number of the question is {len(json_data)}.")
print(f"Selected question is: {json_data[0]['question']}.")
print(f"Selected answer is: {json_data[0]['answer']}.")

The number of calendar data is 20.
The number of the question is 46.
Selected question is: How many meetings do I have attended in total?.
Selected answer is: 18.


In [160]:
print(type(calendar_data['attendees'][0]))

<class 'str'>


## Convert the calendar data

In [161]:
# convert the start and end time into datetime format (%Y-%m-%d %H:%M:%S)
def convert_date(calendar_data):   
    # Convert 'start' and 'end' columns to datetime objects
    calendar_data['start'] = pd.to_datetime(calendar_data['start'])
    calendar_data['end']   = pd.to_datetime(calendar_data['end'])
    # Format 'start' and 'end' columns into the desired string format
    calendar_data['start'] = calendar_data['start'].dt.strftime('%Y-%m-%d %H:%M:%S')
    calendar_data['end']   = calendar_data['end'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    calendar_data['start'] = pd.to_datetime(calendar_data['start'])
    calendar_data['end']   = pd.to_datetime(calendar_data['end'])
    
    # calendar_data['attendees'] = [ str(item) for item in calendar_data['attendees'] ]

    return calendar_data

calendar_data = convert_date(calendar_data)
print(calendar_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ID         20 non-null     object        
 1   status     20 non-null     object        
 2   summary    20 non-null     object        
 3   start      20 non-null     datetime64[ns]
 4   end        20 non-null     datetime64[ns]
 5   duration   20 non-null     int64         
 6   attendees  20 non-null     object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 1.2+ KB
None


In [162]:

calendar_data['attendees'] = [ item.replace('\n', '\n') for item in calendar_data['attendees']]
calendar_data

,ID,status,summary,start,end,duration,attendees
0,mambf90j96gb0m3p8u9fo0q7v8,tentative,Topic: Job Application Discussion,2024-02-02 09:00:00,2024-02-02 10:30:00,5400,walter@ninjatech.ai(accepted)\nnatalia@ninjate...
1,mipnc82k87hc1n4q9v10g1r9w9,confirmed,Contract Negotiation,2024-02-05 14:00:00,2024-02-05 15:30:00,5400,walter@ninjatech.ai(accepted)\nnatalia@ninjate...
2,lokdi73l78id2o5r10w11h2s10u,cancelled,Team Introduction,2024-02-15 09:00:00,2024-02-15 09:30:00,1800,walter@ninjatech.ai(accepted)\narash@ninjatech...
3,npqej64m69je3p6s11x12i3t11v,confirmed,Meeting about Project Overview,2024-02-17 14:00:00,2024-02-17 14:30:00,1800,walter@ninjatech.ai(accepted)\narash@ninjatech...
4,oqrfk55n60kf4q7t12y13j4u12w,confirmed,Onboarding Process,2024-02-18 10:00:00,2024-02-18 11:00:00,3600,walter@ninjatech.ai(accepted)\nhr@ninjatech.ai...
5,psrgl46o51lg5r8u13z14k5v13x,cancelled,Software Development Planning,2024-02-20 14:00:00,2024-02-20 15:00:00,3600,walter@ninjatech.ai(accepted)\nteamlead@ninjat...
6,qtrhm37p42mh6s9v14a15l6w14y,confirmed,Code Review,2024-02-27 09:00:00,2024-02-27 10:30:00,5400,walter@ninjatech.ai(accepted)\nseniorengineer@...
7,mambf12j98gb0m3p8u9fo0h6k2,confirmed,Agile Methodology Training,2024-02-29 14:00:00,2024-02-29 14:20:00,1200,walter@ninjatech.ai(accepted)\ntraining@ninjat...
8,masbk72a24cb0a8k9c7jo0e9s6,confirmed,Client’s Requirement Discussion,2024-03-03 10:30:00,2024-03-03 11:00:00,1800,walter@ninjatech.ai(accepted)\nsalesteam@ninja...
9,mazcr34e95dc9m5o8p7no6t2u7,confirmed,Weekly Team Meeting,2024-03-05 14:00:00,2024-03-05 15:00:00,3600,walter@ninjatech.ai(accepted)\nteam@ninjatech....


In [163]:
str1 = [["walter@ninjatech.ai(accepted)","natalia@ninjatech.ai(accepted)"],
        ["walter@ninjatech.ai(accepted)","natalia@ninjatech.ai(needsAction)"],
        ["walter@ninjatech.ai(accepted)","arash@ninjatech.ai(accepted)"],
        ["walter@ninjatech.ai(accepted)","arash@ninjatech.ai(needsAction)"],
        ["walter@ninjatech.ai(accepted)","hr@ninjatech.ai(accepted)"],
        ["walter@ninjatech.ai(accepted)","teamlead@ninjatech.ai(accepted)"],
        ["walter@ninjatech.ai(accepted)","seniorengineer@ninjatech.ai(needsAction)"],
        ["walter@ninjatech.ai(accepted)","training@ninjatech.ai(needsAction)"],
        ["walter@ninjatech.ai(accepted)","salesteam@ninjatech.ai(rejected)"],
        ["walter@ninjatech.ai(accepted)","team@ninjatech.ai(needsAction)"],
        ["walter@ninjatech.ai(accepted)","arash@ninjatech.ai(accepted)"],
        ["walter@ninjatech.ai(accepted)","alice@ninjatech.ai(accepted)"],
        ["walter@ninjatech.ai(accepted)","bob@ninjatech.ai(accepted)"],
        ["walter@ninjatech.ai(accepted)","carrie@ninjatech.ai(accepted)"],
        ["walter@ninjatech.ai(accepted)","david@ninjatech.ai(accepted)"],
        ["walter@ninjatech.ai(accepted)","arash@ninjatech.ai(needsAction)"],
        ["walter@ninjatech.ai(accepted)","alice@ninjatech.ai(needsAction)"],
        ["walter@ninjatech.ai(accepted)","bob@ninjatech.ai(accepted)"],
        ["walter@ninjatech.ai(accepted)","carrie@ninjatech.ai(rejected)"],
        ["walter@ninjatech.ai(accepted)","david@ninjatech.ai(needsAction)"]
]

# calendar_data['attendees'] = [ '\n'.join(item) for item in str1 ]
print('\n'.join(["walter@ninjatech.ai(accepted)","natalia@ninjatech.ai(accepted)"]))

walter@ninjatech.ai(accepted)
natalia@ninjatech.ai(accepted)


In [97]:
(calendar_data['attendees'][0])

'walter@ninjatech.ai(accepted)\\nnatalia@ninjatech.ai(accepted)'

In [22]:
# Filter the dataframe to get the specific meeting by ID
specific_meeting = calendar_data[calendar_data['ID'] == 'masbk72a24cb0a8k9c7jo0e9s6']
# print(specific_meeting)
    # Get the attendees for the specific meeting
if not specific_meeting.empty:
    attendees_count = specific_meeting['attendees'].str.split('\n').apply(len).sum()
else:
    attendees_count = 0
print(type(specific_meeting['attendees']))
print(specific_meeting['attendees'].str.split('\n').apply(len))

<class 'pandas.core.series.Series'>
8    1
Name: attendees, dtype: int64


In [11]:

# MODEL = "gpt-4"
MODEL = "gpt-3.5-turbo"

question_index = 9 # for question test

In [13]:
def get_completion(MODEL, PROMPT):
    if MODEL == "gpt-4":
        # print(f"""Model is: gpt-4""")
        client = OpenAI(
            api_key=os.environ.get("OPENAI_API_KEY"),
        )
    else:
        # print(f"""Model is: gpt-3.5 turbo""")
        client = OpenAI()

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": PROMPT,
            }
        ],
        model=MODEL, 
    )

    # verify the output
    return chat_completion.choices[0].message.content

Use a Chain-of-Thought approach to break down the problem, analyze the date requirements, and then write the code in Python language. 

    For example, the output have the following format:
    ```python
    def query_dataframe(calendar_data):
        
        return answer
    ```

In [14]:
def get_test_prompt(question):
    
    PROMPT = f"""You are provided a calendar dataset. This calendar is a Pandas dataframe named calendar_data, columns = [ID, status, summary, start, end , duration, attendees].
    This DataFrame calendar_data includes all of your meeting schedule. 
    Provide a chain of thought for your reasoning, analyze the date requirement with python package datetime, filter dataframe, and then fill the python function <query_dataframe>. 

    The input have following columns:
    - ID: meeting ID;
    - status: meeting status, including the following status: cancelled, confirmed, tentative; cancelled means that meeting is cancelled.
    - summary: meeting or event topic;
    - start: the start datetime of meeting, date format: '%Y-%m-%d %H:%M:%S'. for example "2024-02-05 12:00:00";
    - end: the start datetime of meeting, date format: '%Y-%m-%d %H:%M:%S', for example "2024-02-05 13:00:00";
    - duration: meeting duration (second);
    - attendees: people who attend the meeting delimited by the line terminator within 1 sentence.

    **instruction**
    - step 1: analyze if there are any date requirement to query this dataframe, for example, today, this week, next weekend. If true, go to step 2, otherwise go to step 5;
    - step 2: define the time range forthe time requirement based on the provided variable 'today_date';
    - step 3: convert and change the date column 'start' and 'end' from str format to datetime format;
    - step 4: filter the dataframe to include only meetings for the requirement date using python package datetime;
    - step 5: filtering dataframe to get the correct answer using python package pandas;

    Ensure any code you provide can be executed with all required input and variables defined.
    The output should be a markdown python code snippet formatted in the following schema, including the leading and trailing "```python" and "```":
    ```python
    def query_dataframe(calendar_data):
        # get today's date, if the question is related to datetime, use today_date, otherwise ignore this code
        today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')

        <your code here>
        
        return answer
    ```

    Here is the question: {question} 
    """
    return PROMPT



In [15]:
def get_prompt(question):
    
    PROMPT = f"""You are provided a calendar dataset. This calendar is a Pandas dataframe named calendar_data, columns = [ID, status, summary, start, end , duration, attendees].
    This DataFrame calendar_data includes all of your meeting schedule. 
    Provide a chain of thought for your reasoning, analyze the date requirement with python package datetime, filter dataframe, and then fill the python function <query_dataframe>. 

    The input have following columns:
    - ID: meeting ID;
    - status: meeting status, including the following status: cancelled, confirmed, tentative; cancelled means that meeting is cancelled.
    - summary: meeting or event topic;
    - start: the start datetime of meeting, date format: '%Y-%m-%d %H:%M:%S'. for example "2024-02-05 12:00:00";
    - end: the start datetime of meeting, date format: '%Y-%m-%d %H:%M:%S', for example "2024-02-05 13:00:00";
    - duration: meeting duration (second);
    - attendees: people who attend the meeting delimited by the line terminator within 1 sentence.

    **instruction**
    - step 1: analyze if there are any date requirement to query this dataframe, for example, today, this week, next weekend. If true, go to step 2, otherwise go to step 5;
    - step 2: define the time range forthe time requirement based on the provided variable 'today_date';
    - step 3: convert and change the date column 'start' and 'end' from str format to datetime format;
    - step 4: filter the dataframe to include only meetings for the requirement date using python package datetime;
    - step 5: filtering dataframe to get the correct answer using python package pandas;


    Ensure any code you provide can be executed with all required input and variables defined.
    The output should be a markdown python code snippet formatted in the following schema, including the leading and trailing "```python" and "```":
    ```python
    def query_dataframe(calendar_data):
        # get today's date, if the question is related to datetime, use today_date, otherwise ignore this code
        today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')

        <your code here>
        
        return answer
    ```
    
    For example
    Example 1: you are asked a question: "How many tentative event do I have?". The code you generated should be:
    ```python
    def query_dataframe(calendar_data):
        # get today's date, if the question is related to datetime, use variable today_date, otherwise ignore this variable
        today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')  
    
        # filter the dataframe to include meetings with status is 'tentative'
        num_tentative_meetings = calendar_data[calendar_data['status'] == 'cancelled'].shape[0]
        return num_cancelled_meetings
    ```   
    
    Example 2, you are asked a question: "What is the topic of my last meeting today?". The code you generated should be:
    ```python
    def query_dataframe(calendar_data):
        # get today's date   
        today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')  
          
        # convert 'start' and 'end' columns to datetime format   
        calendar_data['start'] = pd.to_datetime(calendar_data['start'], format='%Y-%m-%d %H:%M:%S')   
        calendar_data['end'] = pd.to_datetime(calendar_data['end'], format='%Y-%m-%d %H:%M:%S')
        
        # filter the dataframe for meetings that occurred on the same day as today_date
        filtered_data = calendar_data[(calendar_data['start'].dt.date == today_date.date())]
        
        # sort the filtered data by 'start' datetime column in descending order  
        sorted_data = filtered_data.sort_values(by='start', ascending=False)
        
        # get the topic of the last meeting  
        answer = sorted_data.iloc[0]['summary']
        
        return answer
    ```
    
    Here is the question: {question} 
    """
    return PROMPT



In [16]:
def get_python_code(llm_reply_with_code):
    _python_code_re_pattern = "```python\n(.*?)```"
    llm_reply_without_code = re.sub(
                _python_code_re_pattern, "", llm_reply_with_code, flags=re.DOTALL
                )
    python_code_list = re.findall(_python_code_re_pattern, llm_reply_with_code, re.DOTALL)
    return python_code_list[0]

## Prompt test - Prompt without reflexion

In [17]:
# About 2.2 mins to run the code

# GPT-4

result_summary = {'success': 0, 'wrong_answer': 0, 'error': 0}
question_answer_summary = {'question': '',
                           'generated_code': '',
                           'true_answer':'', 
                           'answer': ''
                        }

q_a_pair_solution_list_for_json = []
result_prompt = []

q_a_pair = json_data[0]
print(f"""Question: {q_a_pair['question']}\n True answer: {q_a_pair['answer']}""")

# define the generated <query_dataframe> function code 
PROMPT = get_prompt(q_a_pair['question'])
llm_reply_with_code = get_completion(MODEL, PROMPT)
python_code_list = get_python_code(llm_reply_with_code)
print(python_code_list)
exec(python_code_list)
answer = query_dataframe(calendar_data.copy())
if q_a_pair['answer'] == answer:
    print('success')
else:
    print(f"""Wrong answer: {answer}""")

PROMPT = get_test_prompt(q_a_pair['question'])
llm_reply_with_code = get_completion(MODEL, PROMPT)
python_code_list = get_python_code(llm_reply_with_code)
print(python_code_list)
exec(python_code_list)
answer = query_dataframe(calendar_data.copy())
if q_a_pair['answer'] == answer:
    print('success')
else:
    print(f"""Wrong answer: {answer}""")  

Question: How many meetings do I have attended in total?
 True answer: 18
def query_dataframe(calendar_data):
    # filter the dataframe to include meetings with attendees
    total_meetings_attended = calendar_data[calendar_data['attendees'].notnull()].shape[0]
    
    return total_meetings_attended

Wrong answer: 20
def query_dataframe(calendar_data):
    # define the time range for the time requirement based on the provided variable 'today_date'
    today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')
    
    # convert and change the date column 'start' and 'end' from str format to datetime format
    calendar_data['start'] = pd.to_datetime(calendar_data['start'], format='%Y-%m-%d %H:%M:%S')
    calendar_data['end'] = pd.to_datetime(calendar_data['end'], format='%Y-%m-%d %H:%M:%S')
    
    # filtering dataframe to get the correct answer using python package pandas
    total_meetings_attended = calendar_data[calendar_data['attendees'].str.contains('you', 

## Prompt with reflexion

In [125]:
def get_prompt_for_refine(question, generated_code):
    
    PROMPT = f"""Reevaluate your previous generated python code for the question: {question}. For this question, a calendar dataset is provided. This calendar is a Pandas dataframe named calendar_data, columns = [ID, status, summary, start, end , duration, attendees].
    check if the generated code mets the rules step by step, then rewrite the python code to ensure previous generated python code code can be executed with all required input and variables defined.
    
    The input have following columns:
    - ID: meeting ID;
    - status: meeting status, including the following status: cancelled, confirmed, tentative; cancelled means that meeting is cancelled.
    - summary: meeting or event topic;
    - start: the start datetime of meeting, date format: '%Y-%m-%d %H:%M:%S'. for example "2024-02-05 12:00:00";
    - end: the start datetime of meeting, date format: '%Y-%m-%d %H:%M:%S', for example "2024-02-05 13:00:00";
    - duration: meeting duration (second);
    - attendees: people who attend the meeting delimited by the line terminator within 1 sentence.

    **rules**
    - The code is grammarly correct and can be executed;
    - The function name is 'query_dataframe';
    - The input is a dataFrame named 'calendar_data';
    - when count the number of meeting, make sure that meeting status consists of three status: cancelled, confirmed, tentative;
    - if the question is related to datetime, then using the Pandas datetime to deal with datetime;
    - if the question is related to datetime, ensure that both sides of the comparison have the same timezone information or both side have no timezone information.

    
    The new python code should be a markdown python code snippet formatted in the following schema, including the leading and trailing "```python" and "```": Here are several code examples
    
    The new output should be a markdown python code snippet formatted in the following schema, including the leading and trailing "```python" and "```":
    ```python
    def query_dataframe(calendar_data):
        # get today's date, if the question is related to datetime, use today_date, otherwise ignore this code
        today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')

        <your code here>
        
        return answer
    ```
    
    The previous generated code is:
    ```
    {generated_code}
    ```
    """
    return PROMPT



In [126]:
def get_prompt_with_error(question, generated_code, error_info):
    
    PROMPT = f"""You are expert in Python Debugging. For the question {question}, You have to analysis given python <Generated Code> and <Error> and generate new python code that handles the error. 
    For this question, a calendar dataset is provided. This calendar is a Pandas dataframe named calendar_data, columns = [ID, status, summary, start, end , duration, attendees].
    check the generated code, then rewrite the python code to ensure previous generated python code can be executed.
    
    The input pandas dataframe named calendar_data have following columns:
    - ID: meeting ID;
    - status: meeting status, including the following status: cancelled, confirmed, tentative; cancelled means that meeting is cancelled.
    - summary: meeting or event topic;
    - start: the start datetime of meeting, date format: '%Y-%m-%d %H:%M:%S'. for example "2024-02-05 12:00:00";
    - end: the start datetime of meeting, date format: '%Y-%m-%d %H:%M:%S', for example "2024-02-05 13:00:00";
    - duration: meeting duration (second);
    - attendees: people who attend the meeting delimited by the line terminator within 1 sentence.

    *Instructions*:
    - Make sure to generate error free code
    - Generated code is able to handle the error
    - The input is a dataFrame named 'calendar_data';
    
    The new output should be a markdown python code snippet formatted in the following schema, including the leading and trailing "```python" and "```":
    ```python
    def query_dataframe(calendar_data):
        # get today's date, if the question is related to datetime, use today_date, otherwise ignore this code
        today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')

        <your code here>
        
        return answer
    ```
    
    *Generated Code*: {generated_code}
    
    *Error*: {error_info}
    """
    return PROMPT



In [127]:
# Generated Code Check
def code_generate_check(question):
    # code generation
    PROMPT = get_prompt(question)
    llm_reply_with_code = get_completion(MODEL, PROMPT)
    python_code_list = get_python_code(llm_reply_with_code)
    # code check
    PROMPT_refine = get_prompt_for_refine(question, python_code_list)
    llm_refine_reply_with_code = get_completion(MODEL, PROMPT_refine)
    python_checked_code = get_python_code(llm_refine_reply_with_code)
    
    return python_checked_code



def exec_code_check(question, generated_code, error_info):
    PROMPT_reflexion = get_prompt_with_error(question, generated_code, error_info)
    llm_reply_with_code = get_completion(MODEL, PROMPT_reflexion)
    python_reflexion_code = get_python_code(llm_reply_with_code)
    
    return python_reflexion_code


In [115]:

result_summary = {'success': 0, 'wrong_answer': 0, 'error': 0}
question_answer_summary = {'question': '',
                           'generated_code': '',
                           'true_answer':'', 
                           'answer': ''
                        }
q_a_pair_solution_list_for_json = []
result_prompt = []

for i in range(5):
    for q_a_pair in json_data: 
        python_code_list = code_generate_check(q_a_pair['question'])
        
        question_answer_summary['question'] = q_a_pair['question']
        question_answer_summary['true_answer'] = q_a_pair['answer']
        question_answer_summary['generated_code'] = python_code_list
        # print(f"""Question: {q_a_pair['question']}\n True answer: {q_a_pair['answer']}""")

        print(q_a_pair['question'])

        
        try:
            exec(python_code_list)
            answer = query_dataframe(calendar_data.copy())
        except Exception as E:
            question_answer_summary['answer'] = repr(E)
            print(f"""Non-executable code: {E}""")
            result_summary['error'] += 1
            pass
        else:
            try: 
                if q_a_pair['answer'] == answer:
                    question_answer_summary['answer'] = 'success'
                    result_summary['success'] += 1
                else:
                    question_answer_summary['answer'] = repr(answer)
                    print(f"""Wrong answer: {answer}""")
                    result_summary['wrong_answer'] += 1
            except Exception as E:
                question_answer_summary['answer'] = repr(E)
                print(f"""Non-executable code: {E}""")
                result_summary['error'] += 1
                pass   
        
    result_prompt.append(result_summary.copy())
    q_a_pair_solution_list_for_json.append(question_answer_summary.copy())
    print(result_summary)

# save the result into JSON file.
# print(json.dumps(result_list, indent=4))
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(result_list, f, indent=4)


How many meetings do I have attended in total?
Wrong answer: 17
How many events do I have scheduled for today?
Wrong answer: 2
How many attendees are there for the meeting with ID masbk72a24cb0a8k9c7jo0e9s6?
Wrong answer: 1
What is the longest meeting on my calendar, tell me the meeting ID?
How many events with duration longer than 1 hours?
How many meeting were cancelled in total?
How many events are scheduled today?
How many events are scheduled tomorrow?
Do I have me with natalia tomorrow, True or False?
How many meeting did I have yestaerday ?
How many meetings do I have for next 3 days?
Wrong answer: 3
What is the topic of my last meeting today?
Wrong answer: Security Best Practices
What is the topic of my next meeting?
Wrong answer: UI/UX Design
What is the minimum meeting time??
How many confirmed meetings are scheduled for tomorrow?
What is the average meeting duration?
What is my latest scheduled meeting for today? tell me meeting ID 
Wrong answer: maddn84w35db8y3u8n2jo0a5y1
W

<string>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


How many meetings which are yet to start this week?
Find the start time of next meeting?
Wrong answer: 2024-04-02 09:30:00
Find the end time of next meeting?
Wrong answer: 2024-04-02 11:00:00
Find the duration of meeting with id 'mahsm54f88qb0j5o8p3uo0a9v8'.?
How many meetings got confirmed?
{'success': 55, 'wrong_answer': 34, 'error': 3}
How many meetings do I have attended in total?
Wrong answer: 20
How many events do I have scheduled for today?
Wrong answer: 2
How many attendees are there for the meeting with ID masbk72a24cb0a8k9c7jo0e9s6?
Wrong answer: 1
What is the longest meeting on my calendar, tell me the meeting ID?
How many events with duration longer than 1 hours?
How many meeting were cancelled in total?
How many events are scheduled today?
How many events are scheduled tomorrow?
Do I have me with natalia tomorrow, True or False?
How many meeting did I have yestaerday ?
How many meetings do I have for next 3 days?
What is the topic of my last meeting today?
Non-executable c

# Prompt with reflexion and error rewrite

In [132]:

result_summary = {'success': 0, 'wrong_answer': 0, 'error': 0}
question_answer_summary = {'question': '',
                           'generated_code': '',
                           'true_answer':'', 
                           'answer': ''
                        }
q_a_pair_solution_list_for_json = []
result_prompt = []


for q_a_pair in json_data: 
    python_code_list = code_generate_check(q_a_pair['question'])
    
    question_answer_summary['question'] = q_a_pair['question']
    question_answer_summary['true_answer'] = q_a_pair['answer']
    question_answer_summary['generated_code'] = python_code_list
    # print(f"""Question: {q_a_pair['question']}\n True answer: {q_a_pair['answer']}""")

    print(q_a_pair['question'])
    
    iter = 0
    error_index = True
    while(error_index and iter<3):
        iter += 1
        try:
            exec(python_code_list)
            answer = query_dataframe(calendar_data.copy())
            if (q_a_pair['answer'] == answer or True):
                error_index = False
                print(f"""Pass {iter}""")
        except Exception as E:
            print(f"""Recheck - Non-executable code type error: {E}""")
            python_code_list = exec_code_check(q_a_pair['question'], python_code_list, repr(E))
            pass
    
    try:
        exec(python_code_list)
        answer = query_dataframe(calendar_data.copy())
    except Exception as E:
        question_answer_summary['answer'] = repr(E)
        print(f"""Non-executable code: {E}""")
        result_summary['error'] += 1
        pass
    else:
        try: 
            if q_a_pair['answer'] == answer:
                question_answer_summary['answer'] = 'success'
                result_summary['success'] += 1
            else:
                question_answer_summary['answer'] = repr(answer)
                print(f"""Wrong answer: {answer}""")
                result_summary['wrong_answer'] += 1
        except Exception as E:
            question_answer_summary['answer'] = repr(E)
            print(f"""Non-executable code: {E}""")
            result_summary['error'] += 1
            pass   
    
    result_prompt.append(result_summary.copy())
    # q_a_pair_solution_list_for_json.append(question_answer_summary.copy())
print(result_summary)

# save the result into JSON file.
# print(json.dumps(result_list, indent=4))
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(result_list, f, indent=4)
    

How many meetings do I have attended in total?
Pass 1
How many events do I have scheduled for today?
Pass 1
How many attendees are there for the meeting with ID 'masbk72a24cb0a8k9c7jo0e9s6'?
Pass 1
Wrong answer: 1
What is the longest meeting on my calendar, tell me the meeting ID?
Pass 1
How many events with duration longer than 1 hours?
Pass 1
How many meeting were cancelled in total?
Pass 1
How many events are scheduled today?
Pass 1
How many events are scheduled tomorrow?
Pass 1
Do I have me with natalia tomorrow, True or False?
Pass 1
How many meeting did I have yestaerday ?
Pass 1
How many meetings do I have for next 3 days?
Pass 1
Wrong answer: 3
What is the topic of my previous meeting today?
Pass 1
Wrong answer: Database Architecture 
What is the topic of my next meeting?
Pass 1
Wrong answer: Database Architecture 
What is the minimum meeting time??
Pass 1
How many confirmed meetings are scheduled for tomorrow?
Pass 1
What is the average meeting duration?
Pass 1
Wrong answer: 3

<string>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


What is my latest scheduled meeting for today? tell me meeting ID 
Pass 1
Wrong answer: maddn84w35db8y3u8n2jo0a5y1
What is the total meeting duration for today?
Pass 1
How many meetings that last more than 2 hours?
Pass 1
Do I have meeting in next 2 hours, True or False?
Pass 1
Wrong answer: False
Do I have meeting in today afternoon, True or False?
Pass 1
How many meetings do I have meeting in this afternoon?
Pass 1
Wrong answer: 11
How many meetings have more than 10 attendees?
Pass 1
How many meetings were scheduled for last weekend?
Pass 1
Wrong answer: 3
How many meetings did I attended last week?
Pass 1
Wrong answer: 1
How many meetings did I attended last three days?
Pass 1
Wrong answer: 2
How many meetings do I have this tuesday?
Pass 1
Wrong answer: 8
Do I have meeting this tuesday, True or False?
Pass 1
Do I have meeting tuesday morning this week, True or False?
Recheck - Non-executable code type error: 'DatetimeProperties' object has no attribute 'week'
Pass 2
How many meeti